In [1]:
import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score
import collections
import pickle
from datetime import datetime

In [2]:
df = pd.read_csv(f"/home/marcin/PycharmProjects/Engineering_Thesis/data_provider/data/CriteoSearchData-sorted-no-duplicates-LabelEncoded.csv"
                 ,sep=','
                ,nrows=1000000
#                 ,skiprows=range(1,2000000)
                )

In [3]:
df = df.rename(columns={'Sale':'sale','SalesAmountInEuro':'sales_amount_in_euro'})

In [4]:
df[:3]

,sale,sales_amount_in_euro,time_delay_for_conversion,click_timestamp,nb_clicks_1week,product_price,product_age_group,device_type,audience_id,product_gender,...,product_category_3,product_category_4,product_category_5,product_category_6,product_category_7,product_country,product_id,product_title,partner_id,user_id
0,0,-1.00000,-1,1596439471,0,0.00,4,7,9599,13,...,609,0,0,0,0,4,1427874,392409,271,8019615
1,0,-1.00000,-1,1596439471,320,0.00,0,3,0,0,...,12,343,0,0,0,10,1541638,375251,130,6209670
2,1,21.24097,965274,1596439471,0,24.99,4,7,0,2,...,0,0,0,0,0,4,1571314,582135,171,8849725


In [119]:
# def convert_timestamp(df):
#     temp = datetime.fromtimestamp(df['click_timestamp'])
#     df['year'] = temp.year
#     df['month'] = temp.month
#     df['day'] = temp.day
#     df['hour'] = temp.hour
#     return df

# df = df.apply(convert_timestamp, axis=1)
# df = df.drop(columns=['click_timestamp'], axis=1)

In [120]:
df[:3]

,sale,sales_amount_in_euro,time_delay_for_conversion,nb_clicks_1week,product_price,product_age_group,device_type,audience_id,product_gender,product_brand,...,product_category_7,product_country,product_id,product_title,partner_id,user_id,year,month,day,hour
0,0.0,-1.00000,-1.0,0.0,0.00,4.0,7.0,9599.0,13.0,48223.0,...,0.0,4.0,1427874.0,392409.0,271.0,8019615.0,2020.0,8.0,3.0,9.0
1,0.0,-1.00000,-1.0,320.0,0.00,0.0,3.0,0.0,0.0,38885.0,...,0.0,10.0,1541638.0,375251.0,130.0,6209670.0,2020.0,8.0,3.0,9.0
2,1.0,21.24097,965274.0,0.0,24.99,4.0,7.0,0.0,2.0,54532.0,...,0.0,4.0,1571314.0,582135.0,171.0,8849725.0,2020.0,8.0,3.0,9.0


In [121]:
# names_of_columns_with_ids = ['audience_id', 'device_type',
#        'partner_id', 'product_age_group', 'product_brand',
#        'product_category_1', 'product_category_2', 'product_category_3',
#        'product_category_4', 'product_category_5', 'product_category_6',
#        'product_category_7', 'product_country', 'product_gender', 'product_id',
#        'product_title', 'user_id']

In [122]:
# utworzenia słownika LabelEncoders dla całego zbioru danych
# LabelEncoders_dict = collections.defaultdict(LabelEncoder)


# df[names_of_columns_with_ids] = df[names_of_columns_with_ids].astype(str)
# # Encoding the variable
# df[names_of_columns_with_ids] = df[names_of_columns_with_ids].apply(lambda x: LabelEncoders_dict[x.name].fit_transform(x))

In [123]:
# file = open(
#     f"/home/marcin/PycharmProjects/Engineering_Thesis/build_and_update_model_server/LabelEncoders_dict2.pickle",
#     "wb")
# LabelEncoders_dict = pickle.load(file)

In [124]:
# file = open(
#     f"/home/marcin/PycharmProjects/Engineering_Thesis/build_and_update_model_server/LabelEncoders_dict2.pickle",
#     "rb")
# LabelEncoders_dict = pickle.load(file)

In [6]:
def first_model(x_train, y_train):
#     df[names_of_columns_with_ids] = df[names_of_columns_with_ids].astype(str)
#     df[names_of_columns_with_ids] = df[names_of_columns_with_ids].apply(lambda x: LabelEncoders_dict[x.name].transform(x))
#     df[names_of_columns_with_ids] = df[names_of_columns_with_ids].apply(LabelEncoder().fit_transform)

    
    X_train, X_test, y_train, y_test = train_test_split(x_train, 
                                                    y_train,
                                                    test_size=0.2,
                                                    random_state=0)
    number_of_samples = len(y_train)
    counter = collections.Counter(y_train)
    percent_of_ones = counter[1]/number_of_samples

    model =  SGDClassifier(loss='log', random_state=1, tol=1e-3, max_iter=1000, penalty='l1', alpha=1e-05, n_jobs=-1, class_weight={0:percent_of_ones, 1:1-percent_of_ones})
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    

    print(collections.Counter(y_test))
    print(collections.Counter(y_pred))

    print('balanced_accuracy_score: {0}'.format(balanced_accuracy_score(y_test, y_pred)))
    print('accuracy_score: {0}'.format(accuracy_score(y_test, y_pred)))
    print('Nieprawidłowo sklasyfikowane próbki: %d' % (y_test != y_pred).sum())
    print('classification_report :\n', classification_report(y_test, y_pred))
    confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
    print(confmat)
    
    return model, X_test, y_test
    

In [7]:
def next_models(model, df, LabelEncoders_dict, X_test, y_test):
    df[names_of_columns_with_ids] = df[names_of_columns_with_ids].astype(str)
    df[names_of_columns_with_ids] = df[names_of_columns_with_ids].apply(lambda x: LabelEncoders_dict[x.name].transform(x))
    
    
    model = model.partial_fit(df.loc[:,df.columns != 'sale'], df['sale'])
    
    y_pred = model.predict(X_test)

    print(collections.Counter(y_test))
    print(collections.Counter(y_pred))

    print('balanced_accuracy_score: {0}'.format(balanced_accuracy_score(y_test, y_pred)))
    print('accuracy_score: {0}'.format(accuracy_score(y_test, y_pred)))
    print('Nieprawidłowo sklasyfikowane próbki: %d' % (y_test != y_pred).sum())
    print('classification_report :\n', classification_report(y_test, y_pred))
    confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
    print(confmat)
    return model

In [8]:
print("PIERWSZY MODEL")
x_train = df.loc[:,~df.columns.isin(['sale'])]
y_train = df['sale']
model, X_test, y_test = first_model(x_train, y_train)

PIERWSZY MODEL
Counter({0: 175965, 1: 24035})
Counter({0: 187927, 1: 12073})
balanced_accuracy_score: 0.7511545662575411
accuracy_score: 0.94019
Nieprawidłowo sklasyfikowane próbki: 11962
classification_report :
               precision    recall  f1-score   support

           0       0.94      1.00      0.97    175965
           1       1.00      0.50      0.67     24035

    accuracy                           0.94    200000
   macro avg       0.97      0.75      0.82    200000
weighted avg       0.94      0.94      0.93    200000

[[175965      0]
 [ 11962  12073]]


In [66]:
print("PIERWSZY MODEL")
model, X_test, y_test = first_model(x_train, y_train)


# for i in range(1, 8):
#     print("=========================================")
#     print("MODEL NUMER: ", i+1)
#     df2 = pd.read_csv(f"/home/marcin/PycharmProjects/Engineering_Thesis/data_provider/data/CriteoSearchData-sorted-no-duplicates.csv""
#                  ,sep='\t'
#                 ,nrows=1000000
#                 ,skiprows=range(1, 2000000*i)
#                 )
#     model = next_models(model, df2, LabelEncoders_dict, X_test, y_test)


PIERWSZY MODEL
Counter({0: 175965, 1: 24035})
Counter({0: 193121, 1: 6879})
balanced_accuracy_score: 0.5390681398802813
accuracy_score: 0.87022
Nieprawidłowo sklasyfikowane próbki: 25956
classification_report :
               precision    recall  f1-score   support

           0       0.89      0.97      0.93    175965
           1       0.36      0.10      0.16     24035

    accuracy                           0.87    200000
   macro avg       0.62      0.54      0.55    200000
weighted avg       0.82      0.87      0.84    200000

[[171565   4400]
 [ 21556   2479]]


In [10]:
df3 = pd.read_csv(f"/home/marcin/PycharmProjects/Engineering_Thesis/data_provider/data/CriteoSearchData-sorted-no-duplicates-LabelEncoded.csv"
                 ,sep=','
                ,nrows=1000000
                ,skiprows=range(1,2000000)
                )

In [52]:
df3 = df3.rename(columns={'Sale':'sale','SalesAmountInEuro':'sales_amount_in_euro'})

In [24]:
import json
import time

s = time.time()
for id, row in df[:10000].iterrows():
    sample_json = row.to_json()
#     s = time.time()
    sample_dict = json.loads(sample_json)    
    sample_dict_result = sample_dict.copy()
    sample_dict.pop('sale', None)
    probabilities = model.predict_proba([list(sample_dict.values())])[0].ravel()
    if probabilities[0] > probabilities[1]:
        sample_dict_result['predicted'] = 0
    else:
        sample_dict_result['predicted'] = 1
        
    sample_dict_result['probabilities'] = json.dumps(list(probabilities))
#     model.predict([list(sample_dict.values())])

#     print(time.time()-s)
#     break
    
print(time.time()-s)


2.4056472778320312


In [ ]:
df2 = pd.read_csv(f"/home/marcin/PycharmProjects/Engineering_Thesis/data_provider/data/CriteoSearchData-sorted-no-duplicates.csv"
             ,sep='\t'
             ,nrows=1000
            )
df2[names_of_columns_with_ids] = df2[names_of_columns_with_ids].astype(str)
s2 = time.time()
for index, value in LabelEncoders_dict.items():
    s = time.time()
    LabelEncoders_dict[index].transform(df2[index])
    print(index)
    print(time.time()-s)
print(time.time()-s2)

In [ ]:
df['product_category(7)'] = df['product_category(7)'].astype(str)
df['product_title'] = df['product_title'].astype(str)

In [ ]:
df.iloc[:,6:] = df.iloc[:,6:].apply(LabelEncoder().fit_transform)
df[:3]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,1:], 
                                                    df['Sale'],
                                                    test_size=0.2,
                                                    random_state=0)

In [ ]:
number_of_samples = len(y_train)
counter = collections.Counter(y_train)
percent_of_ones = counter[1]/number_of_samples
percent_of_ones

In [ ]:
# model =  SGDClassifier(loss='log', random_state=1, tol=1e-3, max_iter=1000, penalty='l1', alpha=1e-05, n_jobs=-1)
model =  SGDClassifier(loss='log', random_state=1, tol=1e-3, max_iter=1000, penalty='l1', alpha=1e-05, n_jobs=-1, class_weight={0:percent_of_ones, 1:1-percent_of_ones})
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score
import collections
print(collections.Counter(y_test))
print(collections.Counter(y_pred))

print('balanced_accuracy_score: {0}'.format(balanced_accuracy_score(y_test, y_pred)))
print('accuracy_score: {0}'.format(accuracy_score(y_test, y_pred)))
print('Nieprawidłowo sklasyfikowane próbki: %d' % (y_test != y_pred).sum())
print('classification_report :\n', classification_report(y_test, y_pred))
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
print(confmat)